In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:

import configparser, os

import huggingface_hub, wandb

config = configparser.ConfigParser()

config.read('../../config/config.ini')

SECRETS = config['SECRETS']

# set openai api key

os.environ['OPENAI_API_KEY'] = SECRETS['openapi_key']

wandb.login(key = SECRETS['wandb_key'])

huggingface_hub.login(token = SECRETS['hf_key'])


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: n-suneel89 (n-suneel-duke). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\91939\.netrc
c:\Me\Duke\Classes\Spring '24\AIPI 540 - Deep Learning\Individual Project\workspace\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\91939\.cache\huggingface\token
Login successful


In [3]:
import sys

sys.path.append('../../')

from scripts.summarization import rag_summ, t5_summ, falcon_summ

In [4]:
import os

base_path = "../../data/pdf"

samples = []

pdfs = os.listdir(base_path)

pdf_path = f"{base_path}/{pdfs[5]}"

CHROMA_PATH = pdf_path.split("/")[-1].split(".")[0]

In [5]:
pages = rag_summ.read_pages(pdf_path)

In [6]:
chunks = rag_summ.get_chunks(pdf_path)

In [7]:
db = rag_summ.get_vectordb(chunks, CHROMA_PATH)

c:\Me\Duke\Classes\Spring '24\AIPI 540 - Deep Learning\Individual Project\workspace\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
%%time

summary = rag_summ.gen_summary(chunks[0], db)

c:\Me\Duke\Classes\Spring '24\AIPI 540 - Deep Learning\Individual Project\workspace\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Me\Duke\Classes\Spring '24\AIPI 540 - Deep Learning\Individual Project\workspace\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: total: 5.02 s
Wall time: 7.1 s


In [9]:
summary

{'summary': "Sita concludes her story by stating that only two more months remained for Rama's return. Hanuman and Sita tell the divine story, hoping it will rid people of their ego. Valmiki, the muni, praises Rama's virtues and qualities extensively in his tale.",
 'text': "Sita concluded her story with the statement that two more months remained. Reading Hanuman and Sita telling the divine story themselves? As Vamana got the better of Bali and saved him from his ahankara, may this tale of Rama as told by Hanuman and Sita rid us of the sense of 'I' and 'my'. Sita concluded her story with the statement that two more months remained. Rama. Valmiki has filled pages with the tale of Rama's virtues. The muni is never satiated drinking from the ocean of Rama's qualities. He describes Rama's gifts and graces sometimes directly, sometimes as seen and admired by others. Thus and in many other ways he dwells"}

In [10]:
t5_pipe = t5_summ.prep_pipeline()

In [11]:
pages[2]

'Mythology is an integral part of \nreligion. It is as necessary for religion and \nnational culture as the skin and the \nskeleton that preserve a fruit with its juice \nand its taste. Form is no less essential than \nsubstance. Myth ology and holy figures are \nnecessary for any great culture to rest on \nits stable spiritual foundation and function \nas a life -giving inspiration and guide.  \nLet us keep ever in our minds the fact \nthat it is the Ramayana and the \nMahabharata that bind our vast  numbers \ntogether as one people, despite caste, \nspace and language that seemingly divide \nthem.  \n1. THE CONCEPTION  \nTo the north of the Ganga was the great \nkingdom Kosala, made fertile by the river \nSarayu. Its capital was Ayodhya, built by \nManu, the famous ru ler of the Solar \ndynasty. From Valmiki\'s description of \nthe capital Kosala, it is clear that ancient \nAyodhya was not inferior to our modern \ncities. Even in ancient India city \ncivilisation had reached a high l

In [12]:
%%time

summary = t5_summ.gen_summary(t5_pipe, pages[2])

Token indices sequence length is longer than the specified maximum sequence length for this model (928 > 512). Running this sequence through the model will result in indexing errors


CPU times: total: 1min 22s
Wall time: 20.4 s


In [13]:
summary

'Mythology is an integral part of religion. It is as necessary for any great culture to rest on its stable spiritual foundation and function as a life -giving inspiration and guide. The Ramayana and the Mahabharata bind our vast numbers together as one people, despite caste, space and language that seemingly divide them. The Yagas were something like our present -day State sponsored big scale conferences and exhibitions.'

In [14]:
falcon_pipe = falcon_summ.prep_pipeline()

In [15]:
%%time

summary = falcon_summ.gen_summary(falcon_pipe, pages[2])

Token indices sequence length is longer than the specified maximum sequence length for this model (928 > 512). Running this sequence through the model will result in indexing errors


CPU times: total: 1min 5s
Wall time: 15.8 s


In [16]:
print(summary)

Dasaratha ruled the kingdom from the capital city of Ayodhya . He had fought on the side of the Devas, and his fame spread in the three worlds . The Yaga was a grand affair and the invitees included many of the kings of the day .
